# **Dataset statistics**

In this notebook some insigths on the dataset are presented.

For this thesis study, the dataset is organized according to Talentia's architecture.
So, I've considered the following tables:
- *TF_QST_QUESTIONNAIRES*, describing the existing questionnaires;
- *TF_QST_QUESTIONS*, describing the questions defined for each questionnaire;
- *TF_QST_QUESTION_TYPES*, describing the possible questions' type;
- *TF_QST_ANSWERS*, describing the possible answers for each defined question.

In [1]:
import os
import sys
sys.path.append('\\'.join(os.getcwd().split('\\')[:-1])+'\\src')

from src.data.TFQuestionnairesDataset import TFQuestionnairesDataset
import src.visualization.visualize as vis

## **Data**

The dataset is managed by the *TFQuestionnairesDataset* class, whose properties describes all the needed information.

First of all, below I've reported just few instances for each table, showing just the "essential" columns. Indeed, all the extracted tables report a set of fields that do not affect directly this task (e.g. the so called audit columns, or those related to the UI).

In [2]:
dataset = TFQuestionnairesDataset()
dataset.load_data(project_root=os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [3]:
print(f"Questionnaires in dataset: {len(dataset.questionnaires)}")

Questionnaires in dataset: 79


In [4]:
vis.get_head_using_essential_columns(dataset.questionnaires, dataset.ESSENTIAL_COLUMNS_QUESTIONNAIRES)

,ID,CODE,NAME
0,1146002,Stress Survey,Stress Survey
1,1146004,Mood,Mood
2,1173001,REMOTE_WORKING,Remote Working - Tell us your experience
3,1173002,EXIT_INTERVIEW,Exit interview - Tell us your experience
4,1173003,OQ,Onboarding Questionnaire


In [5]:
vis.get_head_using_essential_columns(dataset.question_types, dataset.ESSENTIAL_COLUMNS_QUESTION_TYPES)

,ID,CODE,NAME,DESCRIPTION
0,1,SingleChoice,Single choice,Use this type of question to choose one answer...
1,2,MultiChoice,Multi choice,Use this type of question to choose one or mor...
2,3,RatingScale,Rating scale,Use this type of question to rate something.
3,4,Comment,Comment,Use this type of question to aquire feedback.
4,5,ReorderItems,Reorder items,Use this type of question to reorder items.


In [6]:
vis.get_head_using_essential_columns(dataset.questions, dataset.ESSENTIAL_COLUMNS_QUESTIONS)

,ID,TYPE_ID,QUESTIONNAIRE_ID,CODE,NAME,DISPLAY_ORDER
0,1146002,3,1146002.0,1,I often feel overwhelmed by tasks which I'm no...,1
1,1146003,3,1146002.0,2,I'm not comfortable with my working Time sched...,2
2,1146004,3,1146002.0,3,I'm often frustrated because I'm allocated on ...,3
3,1146005,3,1146002.0,4,I often work overtime and I don't have time fo...,4
4,1146006,3,1146002.0,5,The place where I use to work for most of my t...,5


In [7]:
vis.get_head_using_essential_columns(dataset.answers, dataset.ESSENTIAL_COLUMNS_ANSWERS)

,ID,QUESTION_ID,ANSWER
0,1146002,1146002,Completely Disagree
1,1146003,1146002,2
2,1146004,1146002,3
3,1146005,1146002,4
4,1146006,1146002,5


### **Count null values**

It should be obvious that not all columns are going to be used in the future steps.

Therefore, follows the analysis of the null values for each column in all the extracted tables. This allows us to discard them safely.

In [8]:
questionnaires_null_count = vis.count_null_values_for_columns(dataset.questionnaires)
question_types_null_count = vis.count_null_values_for_columns(dataset.question_types)
questions_null_count = vis.count_null_values_for_columns(dataset.questions)
answers_null_count = vis.count_null_values_for_columns(dataset.answers)

In [9]:
vis.plot_null_values(questionnaires_null_count, "Questionnaires", color="#90AEE4")

In [10]:
vis.plot_null_values(question_types_null_count, "Questions' Types", color="#2DCCD3")

In [11]:
vis.plot_null_values(questions_null_count, "Questions", color="#D0006F")

In [12]:
vis.plot_null_values(answers_null_count, "Answers", color="#372082")

## **Questionnaires insights**

Stated this is just an initial analysis on demo data, I wanted to investigate the data distribution of the "ground truth" data.

Hence, follows the computation of some simple statistics and the plots related to the length distribution of questions and answers.

In [13]:
avg_questions = vis.average_number_per_column(dataset.questions, "QUESTIONNAIRE_ID")
print(f"Average number of questions per questionnaire: {avg_questions}")

Average number of questions per questionnaire: 8


In [14]:
avg_question_types = vis.average_number_per_column(dataset.questions, "TYPE_ID")
print(f"Average number of questions per question types: {avg_question_types}")

Average number of questions per question types: 100


In [15]:
avg_answers = vis.average_number_per_column(dataset.answers, "QUESTION_ID")
print(f"Average number of answers per question: {avg_answers}")

Average number of answers per question: 5


In [16]:
aug_ids = dataset.questionnaires[dataset.questionnaires["TOPIC"].isna()]["ID"]
aug_questionnaires = dataset.questionnaires[dataset.questionnaires["ID"].isin(aug_ids)]
hcm_questionnaires = dataset.questionnaires[~dataset.questionnaires["ID"].isin(aug_ids)]

aug_questions = dataset.questions[dataset.questions["QUESTIONNAIRE_ID"].isin(aug_ids)]
hcm_questions = dataset.questions[~dataset.questions["QUESTIONNAIRE_ID"].isin(aug_ids)]

aug_qst_ids = aug_questions["ID"]
hcm_qst_ids = hcm_questions["ID"]

aug_answers = dataset.answers[dataset.answers["QUESTION_ID"].isin(aug_qst_ids)]
hcm_answers = dataset.answers[~dataset.answers["QUESTION_ID"].isin(hcm_qst_ids)]

In [17]:
avg_aug_questions = vis.average_number_per_column(aug_questions, "QUESTIONNAIRE_ID")
print(f"Average number of AUGMENTED questions per questionnaire: {avg_aug_questions}")

avg_hcm_questions = vis.average_number_per_column(hcm_questions, "QUESTIONNAIRE_ID")
print(f"Average number of TALENTIA HCM questions per questionnaire: {avg_hcm_questions}")

Average number of AUGMENTED questions per questionnaire: 8
Average number of TALENTIA HCM questions per questionnaire: 8


In [18]:
avg_aug_answers = vis.average_number_per_column(aug_answers, "QUESTION_ID")
print(f"Average number of AUGMENTED answers per question: {avg_aug_answers}")

avg_hcm_answers = vis.average_number_per_column(hcm_answers, "QUESTION_ID")
print(f"Average number of TALENTIA HCM answers per question: {avg_hcm_answers}")

Average number of AUGMENTED answers per question: 5
Average number of TALENTIA HCM answers per question: 5


### **Length distribution**

In [19]:
vis.plot_length_distribution(dataset.questions, "NAME", "Questions", color="#D0006F")

In [20]:
vis.plot_length_distribution(aug_questions, "NAME", "Augmented Questions", color="#D0006F")

In [21]:
vis.plot_length_distribution(hcm_questions, "NAME", "Talentia HCM Questions", color="#D0006F")

In [22]:
vis.plot_length_distribution(dataset.answers, "ANSWER", "Answers", color="#372082")

In [23]:
vis.plot_length_distribution(aug_answers, "ANSWER", "Augmented Answers", color="#372082")

In [24]:
vis.plot_length_distribution(hcm_answers, "ANSWER", "Talentia HCM Answers", color="#372082")